# Chapter 4

## Linear Regression

A linear model makes a prediction by simply computing a weighted sum of the input features, plus a constant called the *bias term* (also called the *intercept term*).

$$\hat{y}=\theta_0+\theta_1x_1+\theta_2x_2+...+\theta_nx_n$$

- $\hat{y}$ is the predicted value
- $n$ is the number of features
- $x_i$ is the $\text{i}^{th}$ feature value
- $\theta_j$ is the $\text{j}^{th}$ model parameter (including the bias term $\theta_0$ and the feature weights $\theta_1,\theta_2,...,\theta_n$)

Or in a more consice way: vectorized form
$$\hat{y}=h_\theta(\textbf{x})=\theta^T\cdot \textbf{x}$$

- $\theta$ is the model's parameter vector (a colomn vector), containing the bias term $\theta_0$ and the feature weights $\theta_1$ to $\theta_n$
- $\theta^T$ is the transpose of $\theta$ (a row vector)
- $\textbf{x}$ is the instance's feature vector, containing $x_0$ to $x_n$, with $x_0$ always equal to 1
- $\theta^T\cdot x$ is the dot product of $\theta^T$ and $\textbf{x}$
- $h_\theta$ is the hypothesis function, using the model parameters $\theta$

**MSE cost function**
$$MSE(\theta)=\frac{1}{m}\sum_{i=1}^m(\theta^T\cdot \textbf{x}^{(i)}-y^{(i)})^2$$

Compared with RMSE(root mean square error), it is simpler to minimize the mean square error (much easier to compute the gradient).

### Normal Equation
$$\hat{\theta}=(\textbf{X}^T\cdot \textbf{X})^{-1}\cdot\textbf{X}^T\cdot \textbf{y}$$
- $\hat{\theta}$ is the value of $\theta$ that minimizes the cost function
- $\textbf{y}$ is the vector of target values containing $y^{(1)}$ to $y^{(m)}$

**Computational complexity**  
The Normal Equation computes the inverse of $\textbf{X}^T\cdot\textbf{X}$, which is an $n\times n$ matrix (where $n$ is the number of features). The computational complexity of inverting such a matrix is typically about $O(n^{2.4})$ to $O(n^3)$ (depending on the implementation).  
Therefore, the Normal Equation gets very slow when the number of features grows large (e.g., 100,000).  
    On the positive side, this equation is linear with regards to the number of instances in the training set($O(m)$), so it handles large training sets efficiently, provided they can fit in memory. Also, the computational complexity is linear with regards to both the number of instances you want to make predictions on and the number of features.

### Gradient Descent
Gradient descent is a very generic optimization algorithm capable of finding optimal solutions to a wide range of problems. The general idea of gradient descent is to tweak parameters iteratively in order to minimize a cost function.  
Concretely, you start by filling $\theta$ with random values (this is called *random initialization*), and then you improve it gradually, taking one little step at a time, each step attempting to decrease the cost function (e.g., the MSE), until the algorithm *converges* to a minimum.

An important parameter in gradient descent is the size of the steps, determined by the *learning rate* hyperparameter. If the learning rate is too small, then the algorithm will have to go through many iterations to converge, which will take a long time. On the other hand, if the learning rate is too high, it might make the algorithm diverge, with larger and larger values, failing to find a good solution.

**Gradient descent pitfall**  
Not all cost functions look like nice regular bowls. There may be holes, ridges, plateaus, and all sorts of irregular terrains, making convergence to the minimum very difficult.  
Fotunately, the MSE cost function for a linear regression model happens to be a *convex funciton*, which means that if you pick any two points on the curve, the line segment joining them never crosses the curve. This implies that there are no local minima, just one global minimum. It is also a continuous function with a slope that never changes abruptly. These two facts have a great consequence: gradient descent is guaranteed to approach arbitrarily close the global minimum.  
When using gradient descent, you should ensure that all features have a similar scale, or else it will take much longer to converge.

#### Batch Gradient Descent

*Partial derivatives of the cost function*
$$\frac{\partial}{\partial\theta_j}MSE(\theta)=\frac{2}{m}\sum_{i=1}^m(\theta^T\cdot\textbf{x}^{(i)}-y^{(i)})x_j^{(i)}$$

*Gradient vector of the cost function*
$$\nabla_\theta MSE(\theta)=\begin{bmatrix}\frac{\partial}{\partial\theta_0}MSE(\theta)\\\frac{\partial}{\partial\theta_1}MSE(\theta)\\...\\\frac{\partial}{\partial\theta_n}MSE(\theta)\end{bmatrix}=\frac{2}{m}\textbf{X}^T\cdot(\textbf{X}\cdot\theta-\textbf{y})$$

This formula involves calculations over the full training set $\textbf{X}$, at each gradient descent step. This is why the algorithm is called batch gradient descent: it uses the whole batch of training data at every step. As a result it is terribly slow on very large training sets. However, gradient descent scales well with the number of features; training a linear regression model when there are hundreds of thousands of features is much faster using gradient descent than using the normal equation.

*Gradient descent step*
$$\theta^{(next step)}=\theta-\eta\nabla_\theta MSE(\theta)$$

To find a good learning rate, it's okay to use grid search. However, the number of iterations should be limited, or it may take too long to converge.  
A simple solution is to set a very large number of iterations but to interrupt the algorithm when the gradient vector becomes tiny - that is, when its norm becomes smaller than a tiny number $\epsilon$ (called the *tolerance*) - because this happens when gradient descent has (almost) reached the minimum.

**Convergence Rate**  
When the cost function is convex and its slope does not change abruptly, it can be shown that batch gradient descent with a fixed learning rate has a *convergence rate* of $O(\frac{1}{iteration})$

#### Stochastic Gradient Descent

Stochastic gradient descent just picks a random instance in the training set at every step and computes the gradients based only on that single instance. This makes the algorithm much faster since it has very little data to manipulate at every iteration. It also makes it possible to train on huge training sets, since only one instance needs to be in memory at each iteration.  

**Randomness**  
Instead of gently decreasing until it reaches the minimum, the cost function will bounce up and down, decreasing only on average. Over time it will end up very close to the minimum, but once it gets there it will continue to bounce around, never settling down. So once the algorithm stops, the final parameter values are good, but not optimal.  
When the cost function is very irregular, this can actually help the algorithm jump out of local minima, so stochastic gradient descent has a better chance of finding the global minimum than batch gradient descent does.  
Randomness is good to escape from local optima, but bad because it means that the algorithm can never settle at the minimum. One solution to this dilemma is to gradually reduce the learning rate. The steps start out large (which helps make quick progress and escape local minima), then get smaller and smaller, allowing the algorithm to settle at the global minimum. This process is called *simulated annealing*. The function that determines the learning rate at each iteration is called the *learning schedule*.

By convention we iterate by rounds of *m* iterations; each round is called an *epoch*.

#### Mini-batch Gradient Descent

At each step, instead of computing the gradients based on the full training set (as in Batch GD) or based on just one instance (as in Stochastic GD), Mini-batch GD computes the gradients on small random sets of instances called *mini- batches*. The main advantage of Mini-batch GD over Stochastic GD is that you can get a performance boost from hardware optimization of matrix operations, especially when using GPUs.

*Comparison of algorithms for linear regression*

| Algorithm | Large *m* | Out-of-core support | Large n | Hyperparameters | Scaling required | Scikit-learn |
| --- | --- | --- | --- | --- | --- | --- |
| Normal Equation | Fast | No | Slow | 0 | No | LinearRegression|
| Batch GD | Slow | No | Fast | 2 | Yes | n/a |
| Stochastic GD | Fast | Yes | Fast | $\ge$ 2 | Yes | SGDRegressor |
| Mini-batch GD | Fast | Yes | Fast | $\ge$ 2 | Yes | n/a |

## Polynomial Regression

We can actually use a linear model to fit nonlinear data. A simple way to do this is to add powers of each feature as new features, then train a linear model on this extended set of features. This technique is called *polynomial regression*.  
    When there are multiple features, polynomial regression is capable of finding relationships between features (which is something a plain linear regression model cannot do). This is made possible by the fact that `PolynomialFeatures` also adds all combinations of features up to the given degree.

**Learning Curve**  
Apart from cross-validation, another way to get an estimate of a model's generalization performance is to look at the learning curves: these are plots of the model's performance on the training set and the validation set as a function of the training set size. To generate the plots, simply train the model several times on different sized subsets of the training set. 

**The Bias/Variance Tradeoff**  
A model's generalization error can be expressed as the sum of three very different errors:  
*Bias*  
This part of the generalization error is due to wrong assumptions, such as assumpting that the data is linear when it is actually quadratic. A high-bias model is most likely to underfit the training data.  
*Variance*  
This part is due to the model’s excessive sensitivity to small variations in the training data. A model with many degrees of freedom (such as a high-degree polynomial model) is likely to have high variance, and thus to overfit the training data.  
*Irreducible error*  
This part is due to the noisiness of the data itself. The only way to reduce this part of the error is to clean up the data (e.g., fix the data sources, such as broken sensors, or detect and remove outliers).  
Increasing a model's complexity will typically increase its variance and reduce its bias. Conversely, reducing a model's complexity increases its bias and reduces its variance.

## Regularized Linear Models

### Ridge Regression

Ridge regression (also called *Tikhonov regularization*) is a regularized version of linear regression: a *regularized term* equal to $\alpha\sum_{i=1}^n\theta_i^2$ is added to the cost function. This forces the learning algorithm to not only fit the data but also keep the model weights as small as possible. 

The hyperparameter $\alpha$ controls how much you want to regularize the model. If $\alpha=0$ then ridge regression is just linear regression. If $\alpha$ is very large, then all weights end up very close to zero and the result is a flat line going through the data's mean.

Ridge regression cost function: 

$$J(\theta)=MSE(\theta)+\alpha\frac{1}{2}\sum_{i=1}^n\theta_i^2$$

Note that the bias term $\theta_0$ is not regularized. If we define $\textbf{w}$ as the vector of feature weights ($\theta_1$ to $\theta_n$), then the regularization term is simply equal to $\frac{1}{2}(||\textbf{w}||_2)^2$, where $||\cdot||_2$ represents the $\ell_2$ norm of the weight vector.

Ridge regression closed-form: $\hat{\theta}=(\textbf{X}^T\cdot\textbf{X}+\alpha\textbf{A})^{-1}\cdot\textbf{X}^T\cdot\textbf{y}$

### Lasso Regression

Least absolute shrinkage and selection operator regression (simply called Lasso regression) is another regularized version of linear regression: just like ridge regression, it adds a regularization term to the cost function, but it use the $\ell_1$ norm of the weight vector instead of half the square of the $\ell_2$ norm.

Lasso regression cost function:

$$J(\theta)=MSE(\theta)+\alpha\sum_{i=1}^n|\theta_i|$$

An important characteristic of Lasso regression is that it tends to completely eliminate the weights of the least important features (i.e., set them to zero). Lasso regression automatically performs feature selection and outputs a sparse model (i.e., with few nonzero feature weights).

### Elastic Net

Elastic net is a middle ground between ridge regression and lasso regression. The regularization term is a simple mix of both ridge and lasso’s regularization terms, and you can control the mix ratio *r*.

Elastic net cost function:
$$J(\theta)=MSE(\theta)+r\alpha\sum_{i=1}^n|\theta_i|+\frac{1-r}{2}\alpha\sum_{i=1}^n\theta_i^2$$

### Early Stopping

A very different way to regularize iterative learning algorithms such as gradient descent is to stop training as soon as the validation error reaches a minimum. This is called *early stopping*.

With Stochastic and Mini-batch gradient descent, the learning curves are not so smooth as batch gradient, and it may be hard to know whether you have reached the minimum or not. One solution is to stop only after the validation error has been above the minimum for some time, then roll back the model parameters to the point where the validation error was at a minimum.

## Logistic Regression

Logistic regression is commonly used to estimate the probability that an instance belongs to a particular class. 

Logistic regression model estimated probability
$$\hat{p}=h_{\theta}(\textbf{x})=\sigma(\theta^T\cdot\textbf{x})$$

Logistic function
$$\sigma(t)=\frac{1}{1+exp(-t)}$$

Logistic regression cost function
$$J(\theta)=-\frac{1}{m}\sum_{i=1}^m[y^{(i)}log(\hat{p}^{(i)})+(1-y^{(i)})log(1-\hat{p}^{(i)})]$$

Logistic cost function partial derivatives
$$\frac{\partial}{\partial\theta_j}J(\theta)=\frac{1}{m}\sum_{i=1}^m(\sigma(\theta^T\cdot\textbf{x}^{(i)})-y^{(i)})x_j^{(i)}$$

Just like the other linear methods, logistic regression models can be regularized using $\ell_1$ or $\ell_2$ penalties. Scikit-learn adds an $\ell_2$ penalty by default.

### Softmax Regression

The logistic regression can be generalized to support multiple classes directly, without having to train and combine multiple binary classifiers. This is called softmax regression, or multinomial logistic regression.  
The idea is: when given an instance $\textbf{x}$, the softmax regression model first computes a score $s_k(\textbf{x})$ for each class *k*, then estimates the probability of each class by applying the softmax function (also called the *normalized exponential*) to the scores.

Softmax score for class k
$$s_k(\textbf{x})=\theta_k^T\cdot\textbf{x}$$

Once you have computed the score of every class for the instance $\textbf{x}$, you can estimate the probability $\hat{p_k}$ that the instance belongs to class *k* by running the scores through the softmax function: it computes the exponential of every score, then normalizes them.

Softmax function
$$\hat{p_k}=\sigma(\textbf{s}(\textbf{x})_k)=\frac{\exp(s_k(\textbf{x}))}{\sum_{j=1}^K\exp(s_j(\textbf{x}))}$$

- *K* is the number of classes
- $\textbf{s}(\textbf{x})$ is a vector containing the scores of each class for the instance $\textbf{x}$
- $\sigma(\textbf{s}(\textbf{x}))_k$ is the estimated probability that the instance $\textbf{x}$ belongs to class *k* given the scores of each class for that instance

Softmax regression classifier prediction
$$\hat{y}=\arg\max_k\sigma(\textbf{s}(\textbf{x}))_k=\arg\max_k s_k(\textbf{x})=\arg\max_k(\theta_k^T\cdot\textbf{x})$$

The *argmax* operator returns the value of a variable that maximizes a function. In this equation, it returns the value of *k* that maximizes the estimated probability $\sigma(\textbf{s}(\textbf{x}))_k$.

Cross entropy cost function
$$J(\Theta)=-\frac{1}{m}\sum_{i=1}^m\sum_{k=1}^Ky_k^{(i)}\log(\hat{p_k}^{(i)})$$

$y_k^{(i)}$ is equal to 1 if the target class for the $i^{th}$ instance is *k*; otherwise, it is equal to 0.

Cross entropy gradient vector for class k
$$\nabla_{\theta_k}J(\Theta)=\frac{1}{m}\sum_{i=1}^m(\hat{p_k}^{(i)}-y_k^{(i)})\textbf{x}^{(i)}$$